In [1]:
import pandas as pd
import os
import gc

# --- Paths for CTU-13 dataset ---
dataset_files = {
    '1': 'sensor1/sensor1.binetflow',
    '2': 'sensor2/sensor2.binetflow',
    '3': 'sensor3/sensor3.binetflow',
}
# --- End paths ---

# Tentukan nama folder untuk menyimpan hasil sampel
output_folder = 'sampled_data'

# Buat dictionary kosong untuk menampung semua DataFrame LENGKAP
# Variabel ini akan 'hidup' setelah sel ini dijalankan
dict_of_dataframes = {}

In [2]:
print("Memulai proses: Membaca data & mengambil sampel...")

# 1. Buat folder output jika belum ada
try:
    os.makedirs(output_folder, exist_ok=True)
    print(f"Folder output '{output_folder}' telah disiapkan.")
except Exception as e:
    print(f"Gagal membuat folder '{output_folder}': {e}")


# 2. Loop, baca, simpan ke dict, ambil sampel, dan simpan sampel
for key, filepath in dataset_files.items():
    try:
        print(f"\n--- Memproses Dataset {key} ---")
        print(f"Membaca file: {filepath}...")
        
        # a. Baca file
        df = pd.read_csv(filepath, low_memory=False)
        total_rows = len(df)
        
        # b. Simpan DataFrame LENGKAP ke dictionary
        # Ini memenuhi permintaan Anda untuk mengaksesnya nanti
        dict_of_dataframes[key] = df
        print(f"Dataset {key} ({total_rows} baris) telah dimuat ke memori.")

        # c. Tentukan jumlah sampel
        sample_size = min(100, total_rows)
        if sample_size < 100:
            print(f"Peringatan: File {key} hanya memiliki {total_rows} baris. Mengambil {sample_size} baris.")
        else:
            print(f"Mengambil 100 baris acak dari file {key}...")

        # d. Ambil sampel dari DataFrame yang baru saja dibaca
        sampled_df_botnet_not_spam = df[
            df['Label'].str.contains('botnet', case=False, na=False) &
            ~df['Label'].str.contains('spam', case=False, na=False)
        ].sample(n=25, random_state=42)

        sampled_df_botnet_and_spam = df[
            df['Label'].str.contains('botnet', case=False, na=False) &
            df['Label'].str.contains('spam', case=False, na=False)
        ].sample(n=25, random_state=42)

        sampled_df_no_botnet_no_spam = df[
            ~df['Label'].str.contains('botnet', case=False, na=False) &
            ~df['Label'].str.contains('spam', case=False, na=False)
        ].sample(n=50, random_state=42)

        # Gabungkan ketiga DataFrame menjadi satu
        sampled_df = pd.concat([sampled_df_botnet_not_spam, sampled_df_botnet_and_spam, sampled_df_no_botnet_no_spam], ignore_index=True)

        # Acak urutan baris
        sampled_df = sampled_df.sample(frac=1, random_state=42).reset_index(drop=True)

        # Hapus ketiga variabel yang sudah digabungkan agar tidak memenuhi memori
        del sampled_df_botnet_not_spam
        del sampled_df_botnet_and_spam
        del sampled_df_no_botnet_no_spam

        # Memastikan variabel-variabel sudah dihapus dan memori lebih efisien
        gc.collect()

        # e. Tentukan nama file output dan simpan sampel
        original_folder = os.path.dirname(filepath)  # Dapatkan folder asli dari path file
        folder_name = os.path.basename(original_folder)  # Ambil nama folder seperti '1', '2', dll.
        
        # Buat folder di dalam sampled_data dengan nama folder asal
        folder_output_path = os.path.join(output_folder, folder_name)
        os.makedirs(folder_output_path, exist_ok=True)

        base_filename = os.path.basename(filepath)  # Ambil nama file asli
        output_filepath = os.path.join(folder_output_path, base_filename)  # Gabungkan dengan folder output
        
        sampled_df.to_csv(output_filepath, index=False)
        print(f"Sukses! Sampel disimpan ke '{output_filepath}'")
        
    except FileNotFoundError:
        print(f"PERINGATAN: File tidak ditemukan di '{filepath}'. Dataset {key} akan dilewati.")
    except Exception as e:
        print(f"Gagal memproses file {filepath}: {e}. Dataset {key} akan dilewati.")

print("\n--- Semua proses selesai ---")
print(f"Berhasil memuat {len(dict_of_dataframes)} dataset ke memori.")

Memulai proses: Membaca data & mengambil sampel...
Folder output 'sampled_data' telah disiapkan.

--- Memproses Dataset 1 ---
Membaca file: sensor1/sensor1.binetflow...
Dataset 1 (4895158 baris) telah dimuat ke memori.
Mengambil 100 baris acak dari file 1...
Sukses! Sampel disimpan ke 'sampled_data\sensor1\sensor1.binetflow'

--- Memproses Dataset 2 ---
Membaca file: sensor2/sensor2.binetflow...
Dataset 2 (5998133 baris) telah dimuat ke memori.
Mengambil 100 baris acak dari file 2...
Sukses! Sampel disimpan ke 'sampled_data\sensor2\sensor2.binetflow'

--- Memproses Dataset 3 ---
Membaca file: sensor3/sensor3.binetflow...
Dataset 3 (3885792 baris) telah dimuat ke memori.
Mengambil 100 baris acak dari file 3...
Sukses! Sampel disimpan ke 'sampled_data\sensor3\sensor3.binetflow'

--- Semua proses selesai ---
Berhasil memuat 3 dataset ke memori.


In [3]:
# Cek key (dataset) apa saja yang tersedia di dictionary
print(f"Dataset yang berhasil dimuat ke memori: {list(dict_of_dataframes.keys())}")

# Contoh: Mengakses dan menampilkan 5 baris pertama dari dataset '3' (yang LENGKAP)
try:
    print("\nMenampilkan .head() dari dataset '3' (diambil dari memori):")
    
    # Ini adalah cara Anda mengakses data LENGKAP-nya
    df_5_full = dict_of_dataframes['3']
    
    display(df_5_full.head())
    print(f"Ukuran dataset '3' di memori: {df_5_full.shape}")
    
except KeyError:
    print("Dataset '3' tidak berhasil dimuat (mungkin error atau tidak ditemukan).")

Dataset yang berhasil dimuat ke memori: ['1', '2', '3']

Menampilkan .head() dari dataset '3' (diambil dari memori):


,StartTime,Dur,Proto,SrcAddr,Sport,Dir,DstAddr,Dport,State,sTos,dTos,TotPkts,TotBytes,SrcBytes,Label,ActivityLabel,BotnetName,SensorId
0,2022-07-07 09:00:00,0.000000,icmp,147.32.84.165,0x0303,->,202.103.52.147,0xc413,URP,0.0,NaN,1,190,190,flow=From-Botnet-V44-ICMP,1,rbot,3
1,2022-07-07 09:00:00,0.132290,udp,147.32.84.59,54866,<->,216.121.135.141,56313,CON,0.0,0.0,2,133,72,flow=Background-Established-cmpgw-CVUT,0,-,3
2,2022-07-07 09:00:00,3570.602295,tcp,147.32.84.59,44213,<?>,205.188.10.230,443,PA_PA,0.0,0.0,260,19571,7824,flow=Background-Established-cmpgw-CVUT,0,-,3
3,2022-07-07 09:00:00,3595.741455,ipx/spx,00:15:17:2c:e5:2d,NaN,->,ff:ff:ff:ff:ff:ff,NaN,INT,NaN,NaN,568,52954,52954,flow=Background,0,-,3
4,2022-07-07 09:00:00,3467.739746,tcp,69.63.180.46,80,<?>,147.32.85.124,53493,FPA_FPA,0.0,0.0,345,147540,37595,flow=Background,0,-,3


Ukuran dataset '3' di memori: (3885792, 18)
